In [1]:
import pandas as pd
import json
import boto3
import lancedb

# load data.json, can download from 
# https://raw.githubusercontent.com/banool/auslan_dictionary/master/assets/data/data.json
with open('data.json') as f:
    json_data = json.load(f)

json_data = json_data["data"]
embed_df = pd.json_normalize(json_data,
                            ['sub_entries'],
                            ['entry_in_english'],
                            record_prefix='sub_entries_',
                            max_level=0)
embed_df = embed_df.drop(columns=['sub_entries_regions'])
embed_df = embed_df.drop(columns=['sub_entries_video_links'])
embed_df = embed_df.drop(columns=['sub_entries_keywords'])
embed_df['definitions'] = embed_df['sub_entries_definitions'].apply(lambda x: ';'.join([f"{k}: {'; '.join(v)}" for k, v in x.items()]))
embed_df = embed_df.drop(columns=['sub_entries_definitions'])
embed_df['word_id'] = pd.factorize(embed_df['entry_in_english'])[0] + 1
embed_df = embed_df.rename(columns={'entry_in_english': 'word'})
embed_df.to_csv('embed_df.csv', index=False)

embed_test_df = embed_df.head(10)
embed_test_df.to_csv('embed_test_df.csv', index=False)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# AWS
from dotenv import load_dotenv

load_dotenv()
# from .env file
# AWS_ACCESS_KEY_ID=your_aws_access_key_id
# AWS_SECRET_ACCESS_KEY=your_aws_secret_access_key
# AWS_DEFAULT_REGION="ap-southeast-2"


session = boto3.Session(profile_name='auslan', region_name='ap-southeast-2')
s3 = boto3.client('s3')

In [ ]:
# pydantic schema
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_aws.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import LanceDB

bucket_name = "auslan-data-lake-bucket"
prefix = "silver"
object_key = f"{prefix}/vectorized"
uri = f"s3://{bucket_name}/{object_key}"

# connect locally to any path for the embedded db, i.e. uri="./langchain"
db_conn = lancedb.connect(uri=uri, region="ap-southeast-2")

# change to embed_df.csv for full dataset
loader = CSVLoader('embed_test_df.csv', metadata_columns=['word_id'])

docs = loader.load()

bedrock_client = boto3.client(
    'bedrock-runtime',
    "ap-southeast-2",
)

embeddings = BedrockEmbeddings(
    model_id='amazon.titan-embed-text-v2:0', 
    client=bedrock_client
)

In [ ]:
# run once
db = LanceDB.from_documents(docs, embeddings, connection=db_conn, table_name="vectorstore")

In [ ]:
# get table values (first conn)
table=db._table
table_df = table.to_pandas().head()

In [ ]:
# second connection
db2 = lancedb.connect(uri=uri)
table = db2.open_table("vectorstore") # default table name

# full text search index
# table.create_index(metric="cosine", num_sub_vectors=128, replace=True)
table.create_index(metric="cosine", num_sub_vectors=128, index_type="IVF_HNSW_PQ", replace=True,)
table.create_fts_index("text", use_tantivy=False, language="English", stem=True, replace=True, with_position=True)

In [ ]:
from lancedb.rerankers import LinearCombinationReranker

# reranking for hybrid search
reranker = LinearCombinationReranker(weight=0.7)

event_json = {
  "body": "value1",
}

# get the query string from the event
query = event_json.get("body", "{}")
print(query)

# example search query
embedded_query = embeddings.embed_query(query)
print(embedded_query)


In [ ]:
# rerank limit not based on the row but the metadata's word_id
res = table.search(query_type="hybrid").rerank(reranker=reranker).vector(query).text("test query").limit(20).to_pandas()
print(res)

In [ ]:
# Filter top 10 unique words
unique_res = pd.json_normalize(res["metadata"])
unique_res = unique_res.drop_duplicates(subset=['word_id']).head(5)
unique_res = unique_res.drop(columns=['source','row'])


In [ ]:
out = unique_res.to_json()
out

In [15]:
import io

bucket_name = "auslan-data-lake-bucket"

dest_object_key = "gold/embeddings.csv"

with open('data.json') as f:
    json_data = json.load(f)

json_data = json_data["data"]
df = pd.json_normalize(
    json_data,
    ["sub_entries"],
    ["entry_in_english"],
    record_prefix="sub_entries_",
    max_level=0,
)
df["definitions"] = df["sub_entries_definitions"].apply(
    lambda x: ";".join([f"{k}: {'; '.join(v)}" for k, v in x.items()])
)
df = df.drop(
    columns=[
        "sub_entries_regions",
        "sub_entries_video_links",
        "sub_entries_keywords",
        "sub_entries_definitions",
    ]
)
df = df.rename(columns={"entry_in_english": "word"})
df["word_id"] = pd.factorize(df["word"])[0] + 1

csv_buffer = io.StringIO()
df.to_csv(csv_buffer, index=False)

s3.put_object(
    Bucket=bucket_name,
    Key=dest_object_key,
    Body=csv_buffer.getvalue(),
)

# Parquet file for final data retrieval
db_df = pd.json_normalize(json_data, max_level=1)
db_df["word_id"] = pd.factorize(db_df["entry_in_english"])[0] + 1

# convert columns to string
columns_to_convert = ['sub_entries', 'entry_type', 'categories', 'entry_in_english']
for column in columns_to_convert:
    db_df[column] = db_df[column].astype(str)

parquet_buffer = io.BytesIO()
db_df.to_parquet(parquet_buffer, index=False)

s3.put_object(
    Bucket=bucket_name,
    Key="gold/dictionary.parquet",
    Body=parquet_buffer.getvalue(),
)

{'ResponseMetadata': {'RequestId': 'W1HDHDJ17KDVWSWP',
  'HostId': 'aHxYdRXkByup5iBCBkwW7F8r4QZOlPppIo5QvHDVAVX2R8hH/6gXigq+YKc/HIcK+vILPH6le98wMKvjiAQb6jSmTnYx0jh0kSFE0bF1DH4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'aHxYdRXkByup5iBCBkwW7F8r4QZOlPppIo5QvHDVAVX2R8hH/6gXigq+YKc/HIcK+vILPH6le98wMKvjiAQb6jSmTnYx0jh0kSFE0bF1DH4=',
   'x-amz-request-id': 'W1HDHDJ17KDVWSWP',
   'date': 'Thu, 28 Nov 2024 12:15:28 GMT',
   'x-amz-version-id': 'uIz_hAUn3JzrxbD5qzl7eU.7wvuxMXBv',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"4b9640e83a03f959d6f1c0c0d56a3c41"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"4b9640e83a03f959d6f1c0c0d56a3c41"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'uIz_hAUn3JzrxbD5qzl7eU.7wvuxMXBv'}

In [13]:
rs = [1, 2, 3, 4, 5]
parquet_object_key = "gold/dictionary.parquet"

def load_parquet_to_df() -> pd.DataFrame:
    obj = s3.get_object(Bucket=bucket_name, Key=parquet_object_key)
    parquet_data = io.BytesIO(obj["Body"].read())
    df = pd.read_parquet(parquet_data)
    return df

new_df = load_parquet_to_df()
filtered_df = new_df[new_df["word_id"].isin(rs)]

In [14]:
filtered_df

,entry_in_english,sub_entries,entry_type,categories,word_id
0,"a, A",[{'video_links': ['https://media.auslan.org.au...,WORD,[],1
1,abattoir,[{'video_links': ['https://object-store.rc.nec...,WORD,['Work'],2
2,abbreviate,[{'video_links': ['https://object-store.rc.nec...,WORD,['Education'],3
3,abbreviation,[{'video_links': ['https://object-store.rc.nec...,WORD,['Education'],4
4,abdomen,[{'video_links': ['https://object-store.rc.nec...,WORD,"['Health', 'Body Parts']",5


In [ ]:
from lancedb.pydantic import LanceModel
from typing import List, Dict

db_df = pd.json_normalize(json_data, max_level=1)
db_df["word_id"] = pd.factorize(db_df["entry_in_english"])[0] + 1
db_df.to_csv('file_size.csv')
# convert 'word_id' column to str
rs=unique_res['word_id'].astype(int).to_list()
filtered_df = db_df[db_df['word_id'].isin(rs)]
print(json.dumps(filtered_df.to_dict(orient="records"), indent=2))